In [18]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side
from openpyxl.styles import PatternFill
from openpyxl.styles import Alignment, Border, Side, PatternFill
from openpyxl.styles import Font


green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")


thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

def draw_border(ws, start_row, end_row, col):
    start_row = 1
    end_row = 4
    col = 1  # kolumna A

    for row in range(start_row, end_row + 1):
        cell = ws.cell(row=row, column=col)
        cell.border = Border(
            top=thin_border if row == start_row else None,
            bottom=thin_border if row == end_row else None,
            left=thin_border,
            right=thin_border
        )

def draw_merged_row_border(ws, row, start_col, end_col):
    
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)
        cell.border = Border(
            top=thin_border,
            bottom=thin_border,
            left=thin_border if col == start_col else None,
            right=thin_border if col == end_col else None
        )
# Wczytaj dane

start_date = '2025-01-21'
start_date = '2024-01-01'
start_date = '2025-01-01'
start_date= '2025-01-21'
#strategy='0.4'
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h'] # '30m', '15m'
#intervals=['4h']







sheet_output=f'Simulations_summaries- {start_date}.xlsx'


summary_df = pd.read_excel(sheet_output)

# Przygotowanie nowego pliku Excel
wb = Workbook()
wb.remove(wb.active)

# Interwały, które mają pojawić się w tabeli
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

# Unikalne tickery i strategie
tickers = summary_df['Ticker'].unique()
summary_df['Strategy_ID'] = summary_df['Strategy'].astype(str) + ' - ' + summary_df['Description']

strategies = summary_df['Strategy_ID'].unique()
intervals = summary_df['Interval'].unique()
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']
strategies = sorted(strategies, key=lambda x: float(x.split(' - ')[0]))


sheet_output=f"Generated summaries for tickers- {start_date}.xlsx"
for ticker in tickers:
    ws = wb.create_sheet(title=str(ticker))

    # Nagłówek główny
    filt = (summary_df['Ticker'] == ticker) & (summary_df['Interval'] == "1d")
    filtered = summary_df[filt]

    if not filtered.empty:
        start_date = filtered.iloc[0]['Start_date']
        end_date = filtered.iloc[0]['End_date']
        dlugosc=end_date = filtered.iloc[0]['length(d)']
    else:
        start_date = "None"
        end_date = "None"
        dlugosc= "None"
    cell=ws.cell(row=1, column=1).value = f"Analiza dla ticker: {ticker}        Okres: {start_date} - {end_date}    Długość: {dlugosc}(d) .  "
    #cell.font = Font(bold=True)
    cell=ws.cell(row=1, column=1).alignment = Alignment(horizontal='center', vertical='center')
# (opcjonalnie) scal komórki, jeśli chcesz, by tekst był na środku całego wiersza
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=3+len(intervals))
    #draw_merged_row_border(ws, 1, 1, 3+len(intervals))
    cell=ws.cell(row=1, column=1)
    cell.border = thin_border
    # Nagłówek interwałów
    cell=ws.cell(row=2, column=2)
    cell.value = "Interwał"
    cell.border = thin_border
    for i, intv in enumerate(intervals):
        cell=ws.cell(row=2, column=3+i)
        cell.value = intv
        cell.border = thin_border
    cell=ws.cell(row=2, column=3+len(intervals))
    cell.value = "Średnia"
    cell.border = thin_border
    current_row = 3

    for strategy_id in strategies:
        strat_df = summary_df[(summary_df['Ticker'] == ticker) & (summary_df['Strategy_ID'] == strategy_id)]
        if strat_df.empty:
            continue

        # Rozbij strategię na linię tytułu i składniki
        merge_cells=ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+3, end_column=1)
        #draw_border(ws, current_row, current_row+3, 1)
        # Wpisz pełną nazwę strategii do scalonej komórki
        cell = ws.cell(row=current_row, column=1)
        cell.border = thin_border
        cell.value = f"Strategia {strategy_id}"

        # Ustaw zawijanie tekstu i wyśrodkowanie pionowe
        cell.alignment = Alignment(wrap_text=True, vertical='center')

        # Przesuń wskaźnik wiersza o 4

        # Wiersz: No. Trans
        cell=ws.cell(row=current_row, column=2)
        cell.value = "No. Trans"
        cell.border = thin_border
        values = []
        for i, intv in enumerate(intervals):
            val = strat_df[strat_df['Interval'] == intv]['Number'].values
            v = val[0] if len(val) else ''
            cell=ws.cell(row=current_row, column=3+i)
            cell.value = v
            cell.border = thin_border
            if isinstance(v, (int, float)): values.append(v)
        cell=ws.cell(row=current_row, column=3+len(intervals))
        cell.value = round(sum(values)/len(values), 2) if values else ''
        cell.border = thin_border
        current_row += 1

        # Wiersz: PF
        cell=ws.cell(row=current_row, column=2)
        cell.value = "PF"
        cell.border = thin_border
        values = []
        for i, intv in enumerate(intervals):
            val = strat_df[strat_df['Interval'] == intv]['Profit Factor'].values
            v = val[0] if len(val) else ''
            cell= ws.cell(row=current_row, column=3+i)
            cell.value = round(v, 2) if isinstance(v, (int, float)) else v
            cell.border = thin_border
            cell.number_format = '0.00' 
            if isinstance(v, (int, float)): 
                values.append(v)
        cell= ws.cell(row=current_row, column=3+len(intervals))
        cell.value = round(sum(values) / len(values), 2) if values else ''
        cell.border = thin_border
        cell.number_format = '0.00'
        current_row += 1

        # Wiersz: P&L po opłatach
        cell= ws.cell(row=current_row, column=2)
        cell.value = "P&L po opłatach"
        cell.border = thin_border
        values = []
        for i, intv in enumerate(intervals):
            val = strat_df[strat_df['Interval'] == intv]['P&L after fees'].values
            v = val[0] if len(val) else ''
            cell= ws.cell(row=current_row, column=3+i)
            cell.value = round(v) if isinstance(v, (int, float)) else v
            cell.border = thin_border
            cell.number_format = '#,##0'
            if isinstance(v, (int, float)) and v > 0:
                ws.cell(row=current_row, column=3+i).fill = green_fill
            
            if isinstance(v, (int, float)): values.append(v)

        cell= ws.cell(row=current_row, column=3+len(intervals))
        v=round(sum(values) / len(values)) if values else ''
        cell.value = v
        cell.border = thin_border
        cell.number_format = '#,##0'
        if isinstance(v, (int, float)) and v > 0:
            cell.fill = green_fill
        cell.border = thin_border
        current_row += 1  # odstęp

            # Wiersz: Max Drawdown
        cell= ws.cell(row=current_row, column=2)
        cell.value = "Max Drawdown"
        cell.border = thin_border
        values = []
        for i, intv in enumerate(intervals):
            val = strat_df[strat_df['Interval'] == intv]['Max Drawdown'].values
            v = val[0] if len(val) else ''
            cell= ws.cell(row=current_row, column=3+i)
            cell.value = v
            if isinstance(v, (int, float)): values.append(v)
            cell.border = thin_border
        cell= ws.cell(row=current_row, column=3+len(intervals))
        cell.value = round(sum(values)/len(values), 2) if values else ''
        cell.border = thin_border
        ws.column_dimensions['A'].width = 28  # lub dowolna inna wartość
        current_row += 1  # odstęp


# Zapisz wynik
wb.save(sheet_output)

FileNotFoundError: [Errno 2] No such file or directory: 'Simulations_summaries- 2025-01-21.xlsx'